<a href="https://colab.research.google.com/github/zjzsu2000/CMPE258/blob/master/Ungraded_assignment_5/6)_Tesorflow_Redo_6_save_and_load.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Ref:https://www.tensorflow.org/tutorials/keras/save_and_load

# 6) Tesorflow_Redo_6_save_and_load

Model progress can be saved during—and after—training. 


## Installs and imports

In [1]:
!pip install pyyaml h5py  # Required to save models in HDF5 format

In [2]:
import os
import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.2.0-rc4


### Get  dataset


In [0]:
(train_imgs, train_labels), (test_imgs, test_labels) = keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_imgs = train_imgs[:1000].reshape(-1, 28 * 28) / 255.0
test_imgs = test_imgs[:1000].reshape(-1, 28 * 28) / 255.0

## Build model

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.losses import SparseCategoricalCrossentropy as SCC

In [0]:
def build_model():
  model = Sequential([
        Dense(512, activation='relu', input_shape=(784,)),
        Dropout(0.2),
        Dense(10)])

  model.compile(optimizer='adam',loss=SCC(from_logits=True),
                metrics=['accuracy'])
  return model



In [12]:
model = build_model()

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


## Save checkpoints during training

Create a `tf.keras.callbacks.ModelCheckpoint` callback that saves weights only during training:

In [0]:
from tensorflow.keras.callbacks import ModelCheckpoint as mcp

In [0]:
cp_path = "training_1/cp.ckpt"
cp_dir = os.path.dirname(cp_path)

# Create callback to save  the model's weights
cp_callback = mcp(filepath=cp_path,save_weights_only=True,verbose=1)

### Train the model with callback

In [15]:
model.fit(train_imgs,train_labels,epochs=10,
          validation_data=(test_imgs,test_labels),
          callbacks=[cp_callback])  

Epoch 1/10
28/32 [=========================>....] - ETA: 0s - loss: 1.2114 - accuracy: 0.6618
Epoch 00001: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 14ms/step - loss: 1.1421 - accuracy: 0.6780 - val_loss: 0.6800 - val_accuracy: 0.8050
Epoch 2/10
27/32 [========================>.....] - ETA: 0s - loss: 0.4224 - accuracy: 0.8796
Epoch 00002: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 5ms/step - loss: 0.4089 - accuracy: 0.8840 - val_loss: 0.5224 - val_accuracy: 0.8320
Epoch 3/10
26/32 [=======================>......] - ETA: 0s - loss: 0.2892 - accuracy: 0.9315
Epoch 00003: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 5ms/step - loss: 0.2835 - accuracy: 0.9310 - val_loss: 0.4860 - val_accuracy: 0.8410
Epoch 4/10
25/32 [======================>.......] - ETA: 0s - loss: 0.2216 - accuracy: 0.9513
Epoch 00004: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 

### Create collection of TensorFlow checkpoint files that are updated at the end of each epoch

In [17]:
!ls {cp_dir}

checkpoint		     cp.ckpt.data-00001-of-00002
cp.ckpt.data-00000-of-00002  cp.ckpt.index


###Create a new, untrained model.

> Note:you must have a model with the same architecture as the original model.



In [0]:
model = build_model()



In [19]:
# Evaluate the model
loss, acc = model.evaluate(test_imgs,  test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 2.3831 - accuracy: 0.0810
Untrained model, accuracy:  8.10%


###Load the weights from the checkpoint and re-evaluate:

In [21]:
# Loads the weights
model.load_weights(cp_path)

# Re-evaluate the model
loss,acc = model.evaluate(test_imgs,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4174 - accuracy: 0.8620
Restored model, accuracy: 86.20%


It is much better(86.2%) than untrain model(8.1%)

### Checkpoint callback options



In [23]:
# Include the epoch in the file name (uses `str.format`)
cp_path = "training_2/cp-{epoch:04d}.ckpt"
cp_dir = os.path.dirname(cp_path)

# Create a callback that saves the model's weights every 5 epochs
cp_callback = mcp(filepath=cp_path, verbose=1, 
                  save_weights_only=True,period=5)

# Create a new model instance
model = build_model()

# Save the weights using the `checkpoint_path` format
model.save_weights(cp_path.format(epoch=0))

# Train the model with the new callback
model.fit(train_imgs,train_labels,epochs=50,
          callbacks=[cp_callback],
          validation_data=(test_imgs,test_labels),
          verbose=0)


Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


Result checkpoints.

In [25]:
!ls {cp_dir}

checkpoint			  cp-0025.ckpt.data-00001-of-00002
cp-0000.ckpt.data-00000-of-00002  cp-0025.ckpt.index
cp-0000.ckpt.data-00001-of-00002  cp-0030.ckpt.data-00000-of-00002
cp-0000.ckpt.index		  cp-0030.ckpt.data-00001-of-00002
cp-0005.ckpt.data-00000-of-00002  cp-0030.ckpt.index
cp-0005.ckpt.data-00001-of-00002  cp-0035.ckpt.data-00000-of-00002
cp-0005.ckpt.index		  cp-0035.ckpt.data-00001-of-00002
cp-0010.ckpt.data-00000-of-00002  cp-0035.ckpt.index
cp-0010.ckpt.data-00001-of-00002  cp-0040.ckpt.data-00000-of-00002
cp-0010.ckpt.index		  cp-0040.ckpt.data-00001-of-00002
cp-0015.ckpt.data-00000-of-00002  cp-0040.ckpt.index
cp-0015.ckpt.data-00001-of-00002  cp-0045.ckpt.data-00000-of-00002
cp-0015.ckpt.index		  cp-0045.ckpt.data-00001-of-00002
cp-0020.ckpt.data-00000-of-00002  cp-0045.ckpt.index
cp-0020.ckpt.data-00001-of-00002  cp-0050.ckpt.data-00000-of-00002
cp-0020.ckpt.index		  cp-0050.ckpt.data-00001-of-00002
cp-0025.ckpt.data-00000-of-00002  cp-0050.ckpt.index


In [0]:
from tensorflow.train import latest_checkpoint as latest_cp

In [33]:
latest = latest_cp(cp_dir)
latest

'training_2/cp-0050.ckpt'



> Note: the default tensorflow format only saves the 5 most recent checkpoints.





####Reset the model and load the latest checkpoint

In [34]:
# Create a new model instance
model = build_model()

# Load the previously saved weights
model.load_weights(latest)

# Re-evaluate the model
loss, acc = model.evaluate(test_imgs,  test_labels, verbose=2)
print("Restored latest model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4899 - accuracy: 0.8770
Restored latest model, accuracy: 87.70%


It is 87.7% accuracy.

## What are these files?

The above code stores the weights to a collection of [checkpoint](https://www.tensorflow.org/guide/saved_model#save_and_restore_variables)-formatted files that contain only the trained weights in a binary format. Checkpoints contain:
* One or more shards that contain your model's weights.
* An index file that indicates which weights are stored in a which shard.



## Manually save weights

You saw how to load the weights into a model. Manually saving them is just as simple with the `Model.save_weights` method. By default, `tf.keras`—and `save_weights` in particular—uses the TensorFlow [checkpoint](../../guide/checkpoint.ipynb) format with a `.ckpt` extension (saving in [HDF5](https://js.tensorflow.org/tutorials/import-keras.html) with a `.h5` extension is covered in the [Save and serialize models](../../guide/keras/save_and_serialize#weights-only_saving_in_savedmodel_format) guide):

In [0]:
# Save the weights
model.save_weights('./checkpoints/my_checkpoint')


In [37]:
# Create a new model instance
model = build_model()

# Restore the weights
model.load_weights('./checkpoints/my_checkpoint')

# Evaluate the model
loss,acc = model.evaluate(test_imgs,  test_labels, verbose=2)
print("Restored from saving model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4899 - accuracy: 0.8770
Restored from saving model, accuracy: 87.70%


## Save the entire model

Call [`model.save`](https://www.tensorflow.org/api_docs/python/tf/keras/Model#save) to save a model's architecture, weights, and training configuration in a single file/folder. This allows you to export a model so it can be used without access to the original Python code*. Since the optimizer-state is recovered, you can resume training from exactly where you left off.

Entire model can be saved in two different file formats (`SavedModel` and `HDF5`). It is to be noted that TensorFlow `SavedModel` format is the default file format in TF2.x. However, model can be saved in `HDF5` format.


Saving a fully-functional model is very useful—you can load them in TensorFlow.js ([Saved Model](https://www.tensorflow.org/js/tutorials/conversion/import_saved_model), [HDF5](https://www.tensorflow.org/js/tutorials/conversion/import_keras)) and then train and run them in web browsers, or convert them to run on mobile devices using TensorFlow Lite ([Saved Model](https://www.tensorflow.org/lite/convert/python_api#converting_a_savedmodel_), [HDF5](https://www.tensorflow.org/lite/convert/python_api#converting_a_keras_model_))
. 

### SavedModel format

The SavedModel format is another way to serialize models. Models saved in this format can be restored using `tf.keras.models.load_model` and are compatible with TensorFlow Serving. 



> Ref:[SavedModel guide](https://www.tensorflow.org/guide/saved_model)



In [38]:
# Create and train a new model instance.
model = build_model()
model.fit(train_imgs, train_labels, epochs=5)



Epoch 1/5
32/32 [==============================] - 0s 2ms/step - loss: 1.1468 - accuracy: 0.6750
Epoch 2/5
32/32 [==============================] - 0s 2ms/step - loss: 0.4209 - accuracy: 0.8870
Epoch 3/5
32/32 [==============================] - 0s 2ms/step - loss: 0.2826 - accuracy: 0.9230
Epoch 4/5
32/32 [==============================] - 0s 2ms/step - loss: 0.2059 - accuracy: 0.9560
Epoch 5/5
32/32 [==============================] - 0s 2ms/step - loss: 0.1494 - accuracy: 0.9700


In [39]:
# Save the entire model as a SavedModel.
!mkdir -p saved_model
model.save('saved_model/my_model1') 

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: saved_model/my_model1/assets


The SavedModel format is a directory containing a protobuf binary and a Tensorflow checkpoint. 

In [41]:
# my_model directory
!ls saved_model

# Contains an assets folder, saved_model.pb, and variables folder.
!ls saved_model/my_model1

my_model1
assets	saved_model.pb	variables


Reload a fresh Keras model from the saved model:

In [42]:
new_model = keras.models.load_model('saved_model/my_model1')

# Check its summary
new_model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_5 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


The restored model is compiled with the same arguments as the original model. 

In [43]:
# Evaluate the restored model
loss, acc = new_model.evaluate(test_imgs,  test_labels, verbose=2)
print('Restored form whole saved model, accuracy: {:5.2f}%'.format(100*acc))



32/32 - 0s - loss: 0.4184 - accuracy: 0.8680
Restored form whole saved model, accuracy: 86.80%


In [44]:
print(new_model.predict(test_imgs).shape)

(1000, 10)


### HDF5 format

Save format using the [HDF5](https://en.wikipedia.org/wiki/Hierarchical_Data_Format) standard. 

In [45]:
# Create and train a new model instance.
model = build_model()
model.fit(train_imgs, train_labels, epochs=5)

# Save the entire model to a HDF5 file.
# The '.h5' extension indicates that the model should be saved to HDF5.
model.save('my_model2.h5') 

Epoch 1/5
32/32 [==============================] - 0s 2ms/step - loss: 1.1579 - accuracy: 0.6800
Epoch 2/5
32/32 [==============================] - 0s 2ms/step - loss: 0.4311 - accuracy: 0.8780
Epoch 3/5
32/32 [==============================] - 0s 2ms/step - loss: 0.2845 - accuracy: 0.9290
Epoch 4/5
32/32 [==============================] - 0s 2ms/step - loss: 0.2105 - accuracy: 0.9520
Epoch 5/5
32/32 [==============================] - 0s 2ms/step - loss: 0.1608 - accuracy: 0.9690


In [48]:
!ls 

checkpoints  my_model2.h5  sample_data	saved_model  training_1  training_2


Now, recreate the model from that file:

In [53]:
# Recreate the exact same model, including its weights and the optimizer
new_model2 =keras.models.load_model('my_model2.h5')

# Show the model summary
new_model2.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


Check its accuracy:

In [54]:
loss, acc = new_model.evaluate(test_imgs,  test_labels, verbose=2)
print('Restored form save HDF5 format model, accuracy: {:5.2f}%'.format(100*acc))

32/32 - 0s - loss: 0.4184 - accuracy: 0.8680
Restored form save HDF5 format model, accuracy: 86.80%


 This technique saves everything:


> * The weight values
> * The model's architecture
> * The model's training configuration(what you passed to compile)
> * The optimizer and its state, if any (this enables you to restart training where you left)



### Saving custom objects

The key difference between HDF5 and SavedModel is that HDF5 uses object configs to save the model architecture, while SavedModel saves the execution graph. Thus, SavedModels are able to save custom objects like subclassed models and custom layers without requiring the orginal code.

To save custom objects to HDF5, you must do the following:

1. Define a `get_config` method in your object, and optionally a `from_config` classmethod.
  * `get_config(self)` returns a JSON-serializable dictionary of parameters needed to recreate the object.
  * `from_config(cls, config)` uses the returned config from `get_config` to create a new object. By default, this function will use the config as initialization kwargs (`return cls(**config)`).
2. Pass the object to the `custom_objects` argument when loading the model. The argument must be a dictionary mapping the string class name to the Python class. E.g. `tf.keras.models.load_model(path, custom_objects={'CustomLayer': CustomLayer})`


